In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install the Libraries
!pip install -q langchain torch transformers sentence-transformers datasets faiss-cpu unstructured chromadb gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [4]:
# Import Libraries
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
from transformers import DistilBertTokenizerFast, DistilBertForMaskedLM
from langchain import HuggingFacePipeline, LLMChain
from langchain.chains import RetrievalQA, StuffDocumentsChain
from langchain.prompts import PromptTemplate
import gradio as gr
import faiss

In [5]:
# Load Documents from Google Drive
loader = DirectoryLoader('/content/drive/MyDrive/preprocessed_fyp')
data = loader.load()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [6]:
# Split Documents into Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
documents = text_splitter.split_documents(data)


In [7]:
# Text Embedding Model
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Create the Vector Store with Chroma
db = Chroma.from_documents(documents, embeddings)

In [9]:
# Test Similarity Search
question = "What is chemical engineering?"
search_docs = db.similarity_search(question)
print(f"Sample Search Result: {search_docs[0].page_content}")

Sample Search Result: objectives of the program 41 general objectives chemical engineering graduates are well equipped for a wide variety of positions as practicing chemical engineers or for graduate studies in engineering and scientific disciplines 5 the program encourages continued learning and professional development by providing the proper knowledge and stimulation in a setting that promotes personal growth the main objectives of the program include 1 provide students with a highquality education that will enable them to adapt to a rapidly changing technical environment 2 produce graduates who will be productive throughout their careers in a wide range of industrial professional and academic environments 3 develop graduates with strong sense of ethics professionalism and the ability to succeed as both individual and team contributors 4 to produce qualified chemical engineers who are instrumental in the development of the industrial sector of the country 5 to carry out research in 

In [10]:
# # Prepare the LLM Model
# tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
# model = GPT2LMHeadModel.from_pretrained("distilgpt2")

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased", output_hidden_states=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [11]:
# Create Text Generation Pipeline with `max_new_tokens`
text_generator = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,  # Adjusted max_length
    max_new_tokens=200  # Controls the number of tokens generated
)

The model 'DistilBertForMaskedLM' is not supported for question-answering. Supported models are ['AlbertForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuestionAnswering', 'BloomForQuestionAnswering', 'CamembertForQuestionAnswering', 'CanineForQuestionAnswering', 'ConvBertForQuestionAnswering', 'Data2VecTextForQuestionAnswering', 'DebertaForQuestionAnswering', 'DebertaV2ForQuestionAnswering', 'DistilBertForQuestionAnswering', 'ElectraForQuestionAnswering', 'ErnieForQuestionAnswering', 'ErnieMForQuestionAnswering', 'FalconForQuestionAnswering', 'FlaubertForQuestionAnsweringSimple', 'FNetForQuestionAnswering', 'FunnelForQuestionAnswering', 'GPT2ForQuestionAnswering', 'GPTNeoForQuestionAnswering', 'GPTNeoXForQuestionAnswering', 'GPTJForQuestionAnswering', 'IBertForQuestionAnswering', 'LayoutLMv2ForQuestionAnswering', 'LayoutLMv3ForQuestionAnswering', 'LEDForQuestionAnswering', 'LiltForQuestionAnswering', 'Llam

In [12]:
# Create HuggingFacePipeline LLM
llm = HuggingFacePipeline(pipeline=text_generator)

In [13]:
# Create Retriever
retriever = db.as_retriever(search_kwargs={"k":4})

In [104]:
prompt_template = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question", "answer"])


In [105]:
# Create the LLM Chain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

In [106]:

# Now, create the StuffDocumentsChain with the correct reference to 'context'
stuff_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"  # Ensure this matches the input_variables in the LLMChain
)


In [107]:
# Combine into RetrievalQA Chain
qa = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=stuff_docs_chain
)

In [108]:
def query_chatbot(question: str) -> str:
    try:
        relevant_docs = db.similarity_search(question, k=4)
        result = llm_chain.run({"question": question, "context": "\n".join([doc.page_content for doc in relevant_docs])})
        # Return only the 'answer' part of the result
        return result['answer']
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [109]:
with gr.Blocks(title="AASTUChat") as demo:
    gr.HTML(value="""
        <div class="header">
            <h1>Welcome to AASTUChat</h1>
        </div>
    """)

    with gr.Row():
        with gr.Column():
            question_input = gr.Textbox(label="Ask a Question", placeholder="Enter your question here...")
            submit_button = gr.Button("Submit")
        with gr.Column():
            answer_output = gr.Textbox(label="Answer", interactive=False)

    gr.HTML(value="""
        <style>
            .header {
                background-color: #f0f0f0;
                padding: 20px;
                text-align: center;
            }
            h1 {
                color: #333;
                font-size: 24px;
            }
        </style>
    """)

    submit_button.click(fn=query_chatbot, inputs=question_input, outputs=answer_output)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e8d226a6dc20a87a43.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
